<a href="https://colab.research.google.com/github/satani99/generative_deep_learning/blob/main/02_02_deep_learning_convolutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import torch.nn as nn
import torch 
import torch.nn.functional as F

convlayer1 = nn.Conv2d(1, 2, kernel_size=3)

In [15]:
class Conv(nn.Module):
  def __init__(self):
    super(Conv, self).__init__()

    self.conv_layer_1 = nn.Conv2d(3, 10, kernel_size=4, stride=2)
    self.conv_layer_2 = nn.Conv2d(10, 20, kernel_size=3, stride=2)  
    self.linear = nn.Linear(980, 10)

  def forward(self, x):
    x = self.conv_layer_1(x)
    x = self.conv_layer_2(x)
    x = torch.flatten(x, 1)
    output = F.softmax(self.linear(x))
    return output

In [16]:
net = Conv()
print(net)

Conv(
  (conv_layer_1): Conv2d(3, 10, kernel_size=(4, 4), stride=(2, 2))
  (conv_layer_2): Conv2d(10, 20, kernel_size=(3, 3), stride=(2, 2))
  (linear): Linear(in_features=980, out_features=10, bias=True)
)


In [42]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()

    self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1)
    self.conv1_bn = nn.BatchNorm2d(32)
    self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=2)
    self.conv2_bn = nn.BatchNorm2d(32)
    self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1)
    self.conv3_bn = nn.BatchNorm2d(64)
    self.conv4 = nn.Conv2d(64, 64, kernel_size=3, stride=2)
    self.conv4_bn = nn.BatchNorm2d(64)
    self.linear1 = nn.Linear(1600, 128)
    self.linear1_bn = nn.BatchNorm1d(num_features=128)
    self.dropout = nn.Dropout(p=0.5)
    self.linear2 = nn.Linear(128, 10)

  def forward(self, x):
    x = self.conv1(x)
    x = F.leaky_relu(self.conv1_bn(x))
    x = F.leaky_relu(self.conv2_bn(self.conv2(x)))
    x = F.leaky_relu(self.conv3_bn(self.conv3(x)))
    x = F.leaky_relu(self.conv4_bn(self.conv4(x)))
    x = torch.flatten(x, 1)
    x = F.leaky_relu(self.linear1_bn(self.linear1(x)))
    x = self.dropout(x)
    output = F.softmax(self.linear2(x))
    return output



In [43]:
model = ConvNet()
print(model)

ConvNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv1_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2))
  (conv2_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2))
  (conv4_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear1): Linear(in_features=1600, out_features=128, bias=True)
  (linear1_bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (linear2): Linear(in_features=128, out_features=10, bias=True)
)


In [28]:
import numpy as np
from torch.nn.functional import one_hot
import torchvision
import torch
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)


trainset = torchvision.datasets.CIFAR10(root='./content', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./content', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=True, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [30]:
for j, data in enumerate(trainloader, 0):
  inputs, labels = data
  print(inputs.shape)
  break

torch.Size([4, 3, 32, 32])


In [46]:
loss_function = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

epochs = 10

final_losses = []

for i in range(epochs):
  epoch_loss = 0.0
  running_loss = 0.0
  
  for j, data in enumerate(trainloader, 0):
    inputs, labels = data 

    optimizer.zero_grad()

    outputs = model(inputs)
    loss = loss_function(outputs, labels)
    # final_losses.append(loss)

    loss.backward()
    optimizer.step()
    
    epoch_loss += outputs.shape[0] * loss.item()

    running_loss += loss.item()

    if j % 1000 == 999:
      print('[%d, %5d] loss: %.3f' % (i + 1, j+1, running_loss/1000))
      running_loss = 0.0

  print(i+1, epoch_loss / len(trainset))

print('Finished Training')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[1,  1000] loss: 2.087
[1,  2000] loss: 2.093
[1,  3000] loss: 2.073
[1,  4000] loss: 2.068
[1,  5000] loss: 2.067
[1,  6000] loss: 2.059
[1,  7000] loss: 2.054
[1,  8000] loss: 2.055
[1,  9000] loss: 2.042
[1, 10000] loss: 2.033
[1, 11000] loss: 2.044
[1, 12000] loss: 2.029
1 2.057800398015976
[2,  1000] loss: 2.017
[2,  2000] loss: 2.019
[2,  3000] loss: 2.022
[2,  4000] loss: 2.022
[2,  5000] loss: 2.015
[2,  6000] loss: 1.999
[2,  7000] loss: 1.999
[2,  8000] loss: 2.000
[2,  9000] loss: 2.001
[2, 10000] loss: 1.987
[2, 11000] loss: 1.996
[2, 12000] loss: 1.999
2 2.005582045955658
[3,  1000] loss: 1.980
[3,  2000] loss: 1.976
[3,  3000] loss: 1.973
[3,  4000] loss: 1.981
[3,  5000] loss: 1.988
[3,  6000] loss: 1.971
[3,  7000] loss: 1.966
[3,  8000] loss: 1.976
[3,  9000] loss: 1.984
[3, 10000] loss: 1.957
[3, 11000] loss: 1.972
[3, 12000] loss: 1.960
3 1.9726641983413695
[4,  1000] loss: 1.954
[4,  2000] loss: 1.955
[4,  3000] loss: 1.949
[4,  4000] loss: 1.955
[4,  5000] loss: 1.

In [47]:
PATH = './cifar_net.pth'
torch.save(model.state_dict(), PATH)

In [48]:
net = ConvNet()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [49]:
dataiter = iter(testloader)
images, labels = next(dataiter)

In [50]:
outputs = net(images)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [51]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}' for j in range(4)))

Predicted:  truck ship  horse deer 


In [52]:
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

GroundTruth:  truck ship  plane car  


In [54]:
correct = 0
total = 0

with torch.no_grad():
  for data in testloader: 
    images, labels = data 
    outputs = net(images) 

    _, predicted = torch.max(outputs.data, 1) 
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print(f"Accuracy of the network on the 10000 test images: {100 * correct // total} %")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on the 10000 test images: 55 %
